In [1]:
###############################################################################

from IPython.core.interactiveshell import InteractiveShell
import jupyternotify
import folium
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
from selenium import webdriver

InteractiveShell.ast_node_interactivity = "all"
get_ipython().register_magics(jupyternotify.JupyterNotifyMagics);
nest_asyncio.apply();

<IPython.core.display.Javascript object>

In [2]:
data = pd.read_parquet('data.parquet')
data = data[['Date','County','Category','Cents','mL']]
data = data.dropna()

# downsample = pd.read_parquet('downsample.parquet')
# downsample = downsample[['Date','County','Category','Cents','mL']]
# downsample = downsample.dropna()
# downsample.describe(include='all').T.fillna('')

In [3]:
## GEOJSON `iowa-counties.json`
## from https://gist.github.com/mcwhittemore/96aaa132af24d3114643

df = data

choropleth_test = pd.pivot_table(
#     df[df.Date.dt.year==2019],
    df,
    index = 'County',
    columns = 'Category',
#     values = ['Cents','mL'],
    values = 'mL',
    aggfunc = 'sum',
    margins=True,
).fillna(0).applymap(int) #.reset_index()
choropleth_ratios = choropleth_test.copy()
for c in choropleth_ratios.columns[:-1]:
    choropleth_ratios[c] = choropleth_ratios[c]/choropleth_ratios.All
choropleth_ratios = choropleth_ratios.reset_index()
choropleth_ratios['Vodka/Whiskey'] = choropleth_ratios.Vodka/choropleth_ratios.Whiskey
# choropleth_ratios['Vodka/Whiskey'].hist();
# plt.show();

iowa_map = folium.Map(location=[42,-94],zoom_start=7)
folium.Choropleth(
    geo_data='iowa-counties.json',
    data=choropleth_ratios,
    columns=['County','Vodka/Whiskey'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Ratio of Volume (mL) Sales Vodka:Whiskey By County',
).add_to(iowa_map)
display(iowa_map)